# Review Notes on Computational Models

## Finite State Automata

A finite state automaton is an abstraction that consists of a set of states, a set of allowable inputs, a transition function, an initial state, and an accepted state.

**Formal definition**: A finite state automaton consists of a tuple
$$
M=(Q, \Sigma, \delta, q_o, F)
$$
Where $Q$ is a set of finite states, $\Sigma$ is the set of allowable inputs, $\delta$ is the transition function that maps that inputs to state changes, $q_o$ is the initial state, and $F$ is the accepted state.

An FSA can be deterministic or non-deterministic (a state may have multiple transitions for one input). *Need to learn more about this*.

## Turnstyle FSA: 

A turnstyle is a simple example of a FSA. The turnstyle has two states $[locked, unlocked]$, two inputs $[Coin, Push]$. The initial state is locked. There is no accepted state, I'll just use an empty set. 

Let's say the turnstyle requires one quarter to unlock. 

I'll make a general FSA class below. 

In [33]:
class FSA:
    def __init__(self, states, inputs, transition_function, initial_state, final_state):
        self.states=states
        self.inputs=inputs
        self.transition_function= transition_function
        self.current_state= initial_state
        self.final_state= final_state

    def transition(self, input_symbol): 
        if input_symbol in self.inputs:
            if (self.current_state, input_symbol) in self.transition_function:
                self.current_state= self.transition_function[(self.current_state, input_symbol)]
                print(f' Transitioned to state: {self.current_state}')
            else:
                print(f' No transition defined for {self.current_state}, {input_symbol}')

        else: 
            print(f' Invalid Input: {input_symbol} must be in {self.states}') 

    def check_accepting(self):
        return self.current_state in self.final_state

    def get_state(self):
        return self.current_state

    

In [34]:
# Now to simulate the turnstyle
# Define the elements of the tuple; Using dictionaries

states= {'locked', 'unlocked'}
inputs= ['coin', 'push']
transition_function= {('locked', 'coin'): 'unlocked', ('unlocked', 'push'): 'locked'}
initial_state= 'locked'
final_state= set() # assume no accepting state

# Instantiate the model
turnstyle= FSA(states, inputs, transition_function, initial_state, final_state)

# Simulation
actions= ['push', 'coin', 'push', 'coin', 'push', 'coin', 'push']

# Print the outcomes
print('\nSimulating turnstyle FSA ... \n')
for action in actions:
    print(f' Action: {action}')
    turnstyle.transition(action)
    print(f' Current State: {turnstyle.get_state()}')
    print('-' * 30)


Simulating turnstyle FSA ... 

 Action: push
 No transition defined for locked, push
 Current State: locked
------------------------------
 Action: coin
 Transitioned to state: unlocked
 Current State: unlocked
------------------------------
 Action: push
 Transitioned to state: locked
 Current State: locked
------------------------------
 Action: coin
 Transitioned to state: unlocked
 Current State: unlocked
------------------------------
 Action: push
 Transitioned to state: locked
 Current State: locked
------------------------------
 Action: coin
 Transitioned to state: unlocked
 Current State: unlocked
------------------------------
 Action: push
 Transitioned to state: locked
 Current State: locked
------------------------------


## Push Down Automata
Extending the model to deal with the problem of inexact change to explore addition of simple stack memory. 

**Formal definition**: A pushdown automaton consists of a tuple with an additional term $\Gamma$ representing the stack symbols.

$$
M=(Q, \Sigma, \delta, \Gamma, q_o, F)
$$

Symbols can be stored in the stack, as determined by existing 'push' and 'pop' rules that modify the symbols in the stack. 

## Turnstyle PDA

The FSA turnstyle, could only take one coin that was the correct amount for unlocking. This is not very practical. To highlight the differences between these models, I'll add a stack memory mechanism to the turnstyle that stores dimes and nickels, with push/pop rules that compute the total amoung of change inserted (up to 25 cents). Just modifying the FSA for this...

In [38]:
# PDA Simulation Class

class PDA:
    def __init__(self, states, inputs, stack_symbols, transition_function, initial_state, final_state):
        self.states=states
        self.inputs=inputs
        self.stack_symbols=stack_symbols
        self.stack= [] # initialize list to hold the stack
        self.transition_function= transition_function
        self.current_state= initial_state
        self.final_state= final_state

    def stack_value(self):
        '''calcultes the value of coins in the stack'''
        value_map+{'N':5, 'D':10, 'Q':25}
        return sum(value_map[coin] for coin in self.stack)

    def transition(self, input_symbol):
        if input_symbol not in self.inputs:
            print(f'invalid input: {input_symbol}')
            return
            
        if (self.current_state, input_symbol) in self.transition_function:
            action= self.transition_function[(self.current_state, input_symbol)]
            # Stack operations
            if action.get('push'):
                self.stack.append(action['push']) # push coin to stack
            if action.get('pop') and self.stack:
                self.stack.pop() # pop from stack

            if self.stack_value() >=25:
                self.current_state= 'unlocked'
                print(f' Turnstyle Unlocked')
            else:
                self.current_state= action['next_state']
            # Clear stack if turnstyle is pushed
            if action.get('clear_stack'):
                self.stack.clear()
            print(f' Action: {input_symbol}, New State: {self.current_state}, Stack: {self.stack} (Total: {self.stack_value()} cents')
        else: 
            print(f' No transition defined for ({self.current_state}, {input_symbol}).')

    def get_state(self):
        ''' return the current state'''
        return self.current_state

    def get_stack(self): 
        ''' return the current stack list'''
        return self.stack
                    
                              
 


In [44]:
# Now to simulate the turnstyle as a PDA

states= {'locked', 'unlocked'}
inputs= {'nickel', 'dime', 'quarter', 'push'}
stack_symbols= {'N', 'D', 'Q'} 
Initial_state= 'locked'
final_state= set()

# Transition function
transition_function= {
    ('Locked', 'nickel'): {'next_state': 'locked', 'push': 'N'},
    ('Locked', 'dime'): {'next_state': 'locked', 'push': 'D'},
    ('Locked', 'quarter'): {'next_state': 'locked', 'push': 'Q'},
    ('Locked', 'push'): {'next_state': 'locked', 'clear_stack': True},
}

# Instantiate the model
Turnstyle_pda= PDA(states, inputs, stack_symbols, transition_function, initial_state, final_state)

# Simulate
actions= [
    'nickel', 'nickel', 'nickel', 'nickel', 'nickel', 'push'
]

print(f' Simulating')
for action in actions: 
    result= PDA.transition(action)
    print(f' Current State: {PDA.get_state()}, Current Stack: {PDA.get_stack()}')
    print("-" * 40)

 Simulating


TypeError: transition() missing 1 required positional argument: 'input_symbol'